# Limpiar barcodes

## Parámetros

In [10]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/data'
FILE_NAME = 'Barcodes Catalog MX.xlsx'

## Entorno

In [12]:
from numpy import nan
from pathlib import Path
from re import sub, UNICODE
from unicodedata import normalize
from pandas import ExcelFile

BASE_DIR = Path(BASE_DIR)

def clean_text(text: str, pattern: str="[^a-zA-Z0-9\s,]", lower: bool=False) -> str: 
    '''
    Limpieza de texto
    '''
    # Reemplazar acentos: áàäâã --> a
    clean = normalize('NFD', str(text).replace('\n', ' \n ')).encode('ascii', 'ignore')
    # Omitir caracteres especiales !"#$%&/()=...
    clean = sub(pattern, ' ', clean.decode('utf-8'), flags=UNICODE)
    # Mantener sólo un espacio
    clean = sub(r'\s{2,}', ' ', clean.strip())
    # Minúsculas si el parámetro lo indica
    if lower: clean = clean.lower()
    # Si el registro estaba vacío, indicar nulo
    if clean in ('','nan'): clean = nan
    return clean

In [3]:
df = ExcelFile(BASE_DIR.joinpath(FILE_NAME)).parse(sheet_name=0)
df.sample()

Archivo con nombre Barcodes Catalog MX.xlsx fue encontrado en:
/Users/efraflores/Desktop/EF/Corner/Catalog/data
Con 471499 renglones y 7 columnas


,Currency,Product ID,Name,Barcodes,Unnamed: 4,Unnamed: 5,Unnamed: 6
362259,MXN,4760574,Correa Chroma L,{7898582507177},NaN,NaN,NaN


In [14]:
aux = df['Barcodes'].map(clean_text).str.split(',', expand=True).set_index(df.index)
aux.sample()

,0,1,2,3,4,5,6,7,8,9
382848,813463010673,None,None,None,None,None,None,None,None,None


In [15]:
aux[aux[1].notnull()]

,0,1,2,3,4,5,6,7,8,9
340,22796940919,22796910912,None,None,None,None,None,None,None,None
342,2521613000000,201070000000,None,None,None,None,None,None,None,None
343,2634626000007,6557690,None,None,None,None,None,None,None,None
345,31009678433,78917892261,None,None,None,None,None,None,None,None
346,3360372078500,3360,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
471439,9999917216004,277240000002,None,None,None,None,None,None,None,None
471444,9999917267082,7500326333132,None,None,None,None,None,None,None,None
471454,9999917271324,7501003313287,7501003305855,None,None,None,None,None,None,None
471460,9999917377910,3178530400687,None,None,None,None,None,None,None,None


In [17]:
aux[aux[1].str[0]=='0']

,0,1,2,3,4,5,6,7,8,9
44974,12993102029,012993102029,None,None,None,None,None,None,None,None
62141,1988824,019200008884,None,None,None,None,None,None,None,None
77778,25500000343,025500000343,None,None,None,None,None,None,None,None
89827,31142000115,031142000115,None,None,None,None,None,None,None,None
89828,31142000696,031142000696,None,None,None,None,None,None,None,None
89845,31142105209,031142105209,None,None,None,None,None,None,None,None
90227,31534153009,031534153009,None,None,None,None,None,None,None,None
104150,400000053820,0400000053820,2599971000000,None,None,None,None,None,None,None
130186,41900079499,041900079499,None,None,None,None,None,None,None,None
136727,44774115546,044774115546,None,None,None,None,None,None,None,None
